# How to create a UPS file in Arches (coal-fired)

## 1. Create the geometry

The intrusions define all the surfaces that are interior to the boundaries of your computational domain.

### Codes

Obtain a copy of intrusion.py, geometry_uintah.py, and object_info.py. The driver is intrusion.py. Geometry objects are defined in geometry_uintah.py. Data files are read in by object_info.py. You will need to modify these codes to represent the geometry and wall boundary conditions for your particular case. For example, you will need to create new geometry objects for the intrusions specific to your system geometry. You may want to create instrusions for all non-refractory heat transfer surfaces even if these objects don't extend into your computational domain. These intrusions will make it easier to visualize the various regimes where wall heat transfer is occurring.

### Inputs

Obtain a copy of intrusion_data.xlsx. This Excel file contains information about the system geometry and wall heat transfer. Update the geometry sheets as needed for your case (overall geometry, various intrusion types such as cooling panels, burner quarl, etc.). These sheets include all information (geometry and temperature) required by the intrusion model. From these updated sheets, create the text files read in by intrusion.py (geometry_info.txt, panel_info.txt, etc.).

### Run

Run intrusion.py. To run python, one option is pycharm from JetBrains. 

### Output

Output is “intrusion.xml.” This file contains all the intrusion information needed to run your case. You need to include this file in your ups file as follows:

> [NO SHOW] <BoundaryConditions>
    <include href="geometry/intrusion.xml" section="intrusions" /> "

### Verify

You may need to iterate on the location and size of intrusions after looking at the Arches geometry in VisIt. 

## 2. Wall heat transfer

This section defines the heat transfer properties for all of the surfaces defined in the simulation (domain boundaries + intrusions). The emissivity model is run to determine ash properties/parameters for the ash deposition model in Arches. 

### Codes

Obtain a copy of wall_model_deposition.py, geometry_uintah.py, and object_info.py; geometry_uintah.py and object_info.py are the same files used for the instrusions. You will need to modify the driver, wall_model_deposition.py, based on the changes you made to intrusion.py.

Download project “ash” from https://gitlab.chpc.utah.edu/groups/ccmscteam. To clone the repository, use “git clone https://gitlab.chpc.utah.edu/ccmscteam/ash_emissivity.git.” Use your UNID & password. On thing to note about this repository. Ben made separate branches for python 2.7 and python 3. The master is on the python 3 branch. So, if you are using python 2.7, make sure to change from the default branch by checking out the python2.7 branch after you have cloned the repo. If you already have downloaded the project, update with “git stash” then “git pull”. 

### Inputs

Obtain a copy of intrusion_data.xlsx. Update the wall heat transfer sheets (“Wall_deposition” & “Wall_dep_global”) as needed with information required by the WallHT model. This information includes tube side temperature, thickness of wall, emissivity of wall and thermal conductivity of wall. From these updated sheets, create the text files read in by wall_model_deposition.py (wall_deposition_info.txt, wall_info.txt, wallht_info.txt).

In the driver for the ash emissivity model, GenericEmissivity.m, change “xi” to match ash composition for your coal. Also change “caseid” to reflect the coal name.

### Run

Run wall_model_deposition.py.

For the ash emissivity, it is easiest to “cd” to the directory where GenericEmissivity.m is located before launching Matlab. Once Matlab is launched, run GenericEmissivity.m. Use Matlab from a CHPC cluster node via FastX. Open a web browser to any of the following servers: "ash1.chpc.utah.edu:3000" "ash2.chpc.utah.edu:3000" "ash3.chpc.utah.edu:3000" "ash4.chpc.utah.edu:3000.” There is a link in the screen that says, “Looking for the Desktop client?” Click on that and install the client. Then you can use the FastX client to connect to ash1, 2, 3 or 4 and run Matlab there (open Xterm, type "module load matlab", “matlab"). Running it this way uses the resources on the server instead of your laptop. Also, it is persistent even if you lose power or network on your laptop. Just reconnect through FastX again. If you have questions about running Matlab, send them to "issues@chpc.utah.edu” and probably Martin Cuma will guide you on the quirks with using Matlab. Also check out https://www.chpc.utah.edu/documentation/software/fastx2.php.

### Output

Output is “wall.xml,” which is included in the ups file as follows:

[NO SHOW]'<BoundaryConditions>
  <include href="geometry/wall.xml" section="WallHT" />'
    
    [NEED TO VERIFY OUTPUT OF EMISSIVITY MODEL WITH MILO]

### Verify

The num_minerals parameter in geometry_uintah->wall_model must be set to the number of mineral entries for the ash deposits (see Section [subsec:Emissivity-model])

## Coal properties, including particle size distribution

Assume Rosin-Rammler distribution for coal particle sizes. Compute weights for the particle size distribution based on fit to Weibull PDF. Then, create the NAME.xml file with all the properties for the coal in your case.

Choose weights that conserve first three moments[???]. 

### Codes

Obtain a copy of PSD.m.

Copy SUFCO.xml to NAME.xml in your run directory. 



### Inputs

You need to specify the particle sizes (diameters) that will be used to transport the moments of the Rosin-Rammler distribution in the simulation.

Rename & update 'NAME.xml' to reflect coal properties & devolatilization/char oxidation parameters that are specific to your coal.See the "Coal_xml" ipython notebook for further information about each of the entries.

### Run

To run the Matlab script PSD.m, follow the directions in the "Wall Heat Transfer" section above for running Matlab.

There is nothing to run with the NAME.xml file.

### Output

The weights of the each of the particle sizes. These particle sizes and weights are included in the NAME.xml file.

## Mixing table

Defining the boundary conditions is an iterative process. You will run the python driver (Driver.py) three times. The first time you run it is to create an input file for the mixing table code.

### Codes

Download project “bc_creation_tool” from https://bitbucket.org/bisaac2/bc_creation_tool. You will need to set up a bitbucket account to access this repository. To clone the repository, use “git clone https://jspinti@bitbucket.org/bisaac2/bc_creation_tool.git”

You will also need a copy of the “table” code (talk to Phil; WE NEED TO GET THIS CODE IN A REPOSITORY THAT PEOPLE CAN DOWNLOAD WITH SOME INSTRUCTIONS ON HOW TO SET IT UP; ADD tools to visualize table? Tools in table/work/python & in bc_creation_tool/Table_Query_v2.py) or better yet, ask Phil to create the table for you.

### Inputs

Change the following lines in Driver.py according to the directory you are running in & file you are using: 

- os.chdir('/Users/u0114312/Google Drive/Bitbucket_project/bc_creation_tool')
- original_ups_file='NAME.ups' 
- new_ups_file='test.ups' # name of new input file that will be created 
- coal_filename='NAME.xml' 
- table_name='NAME_TABLE.mix' 
- file_directory="bc_files/" # where to send created files
- table_base_name='NAME_TABLE.dat'

If you don't have a ups file already set up for your case, copy OFC_original.ups to NAME.ups. 

Open bc_conditions.py. Look for function create_table_input_file. This is the input file for creating a mixing table. Edit the print statments to print out the information that matches your coal. For example:

> 24, 30, 30 !NC(coal gas mix frac), NTX(heat loss), NTZ(mix frac) - How many table entries you want for each independent variable in your table? Ben tested sensitivity to table resolution & found these numbers to be adequate.

[FIX]For 2D table, only NC and NTX will be read.

Specify composition of each gas (oxidant and fuel) stream. Ben assumes all water in coal is in gas phase (oxidant streams). You need to add water from coal to one or all of the gas/oxidant streams, then recompute the composition of the oxidant streams. Ben didn't notice any sensitivity in his QOIs to the water distribution (e.g. all in the gas stream carrying coal or equally distributed among all the gas streams). 

You need to specify two “REACTANTS” entries (see below). Default is mass fraction. For mole fraction, you must specify “MOLE” in column after “G”. Make sure columns match up exactly with what is already in the file.

> REACTANTS 1  
> 300.00 !TMP (unformatted)  
> O 2. 0. 0. 0. O2 0.000 G  
> C 1. O 2. 0. 0. CO2 0.974659924 G   
> H 2. O 1. 0. 0. H2O 0.025340076 G ! (Blank line) 

You also need to find enthalpy for various streams that produces a temperature match for the stream (CODE SNIPPET from Ben???).

### Run

Run Driver.py. You will get the following error: 
> “ERROR! enthalpy of formation for coal in : NAME.xml is different than what was computed in formation_enthalpy()”.

Paste the computed value that is given into your NAME.xml file for raw_coal_enthalpy. The char abstraction has been removed in the current version of Arches, so char_enthalpy will not be used. The ash_enthalpy cancels out because it is non-reacting, so its value can be left at 0.0.

> <raw_coal_enthalpy>-2854798.2724</raw_coal_enthalpy>< !–raw coal enthalpy (at 298.15 K) [J/kg] –>  
> <char_enthalpy>0.0</char_enthalpy><!– char enthalpy (at inlet conditions) [J/kg] –>  
> <ash_enthalpy>0.0</ash_enthalpy><!– ash enthalpy (at inlet conditions) [J/kg] –>

Rerun Driver.py. It may crash again, but at this point, you will have the NAME.dat file. Use this NAME.dat file to generate a mixing table (gas phase chemistry) for your particular coal.

Copy NAME.dat file to “table” directory. Run the "table" code using the NAME.dat and NAME.thm files. 

### Output

The output of the “table” code is a NAME.mix file. You will need this file for the next steps.

Edit mixing table file so that the names of the independent variables match those in NAME.ups file and in bc.xml files (primary, sec, outlet, wall, etc).
> [NO SHOW]<coal eta_label="coal_gas_mix_frac" fp_label="mixture_fraction" hl_label="heat_loss" />

Move NAME.mix file to bc_creation_tool directory. Don't forget to gzip mixing table when you're all finished.

## Boundary conditions

Run Driver.py a second time to create the BC files that will define all of your system boundaries.

### Codes

Same as the codes for the "Mixing table" section

### Inputs

You need xml files for all input streams, primary.xml, secondary.xml, etc. and for all other faces (walls & outlet). Use files in bc_creation_tool as templates and then edit fields to reflect inputs for your case. Remember that moving moisture from coal to oxidant means oxidant flowrates increase while coal feed rate (CFR) decreases. Also, we assume that particle velocity = gas velocity in the stream feeding the coal. 

Add names of all xml files to Grid->Boundary Condition in NAME.ups file. Specify name of file to compute scaling constants from [STREAM WITH COAL??] in NAME.xml. Changing coal CFR changes scaling factors in NAME.ups file, not anything in bc_files.

### Run

Run Driver.py.  

### Output

Script will create BC.xml files in new directory that you specify. These new files will have the orientation in their name & will inlcude BCs for the solid phase (coal particles).



## Computing inlet areas

You will run Driver.py a third time to obtain the inputs needed to generate the exact inlet areas for all velocity inlets. These exact areas are needed to compute particle weights. Because inlets are approximated by stairsteps, theoretical area is not equal to real area. 

### Codes

Same as the codes for the "Mixing table" section

### Inputs

Review NAME.ups file to ensure that the correct geometries are specified for all inlets.

To find out the real area, add the following line in <ARCHES><BoundaryConditions> in the NAME.ups (or TEST.ups) file you specified above.

[NO SHOW]<force_area_calc/>



You should have only one patch specified in your NAME.ups file. You may need to chop off the domain to make the case small enough that it doesn't crash on one patch. 

### Run

Copy the following files to a machine where you have an Arches build: NAME.ups, bc_files (entire directory), NAME.mix, intrusion.xml & wall.xml. Run the case on ONE processor. [WHAT ABOUT part.mix.gz??]

To locate where in the array the areas should be copied into:
> for x in myFace.input_files:  
> (LINES)  
> myBC = BC(x, myFace.names[count])  
> (ADD THIS LINE)  
> print('count=', count, myFace.names[count]) 

### Output

In the output you will get values for all inlet areas. Copy these areas into Driver.py where it says #CORRECT AREAS

Make sure the corrected area matches the area in Arches. The other areas are based on the input file, which may be different than what results due to mesh resolution, and don't matter.

### OPL

Use opl_finder.m from Derek to compute opl. He posted it on Slack. Can change density to match your apparent density. I changed it from 1300 to 1109 & it didn't seem to make a difference.

The part_weight variable should be a vector of your quadrature weights. The units being # of particles / m^3, which is weight_boundary_condition*Weight_scaling_constant from the relevant input file. This is “<Weights>, <scaling_const>”.[NO SHOW]

### Notes on svn repository

Need to get read/write access to this repository: 
    
> svn co https://svn.chpc.utah.edu/repo/uintah/ --username (UNID)

How to import only part of the repo

svn add 'YOUR DIR'   

> svn import /path/to/mytree \ http://svn.example.com/svn/repo/some/project \ -m "Initial import" Adding mytree/foo.c Adding mytree/bar.c Adding mytree/subdir Adding mytree/subdir/quux.h 

svn add  - “add” recursively adds   
svn commit -m “COMMENT”  - “commit” only commits changes  
svn update  
svn log  
svn status

What about tools to check mass balance? Energy balance?
